# Preferential sampling 

In [1]:
import numpy as np
import pandas as pd
import sklearn 
import imblearn
import matplotlib.pyplot as plt 
plt.rc("font", size=14)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from sklearn.metrics import classification_report, confusion_matrix
import math 

In [2]:
taiwan_data = pd.read_csv('taiwan_data.csv')
print(taiwan_data)

       LIMIT_BAL  SEX  EDUCATION  MARRIAGE  Age Group  PAY_0  PAY_2  PAY_3  \
0          20000    1          2         1          0      2      2     -1   
1         120000    1          2         2          0     -1      2      0   
2          90000    1          2         2          0      0      0      0   
3          50000    1          2         1          0      0      0      0   
4          50000    0          2         1          1     -1      0     -1   
...          ...  ...        ...       ...        ...    ...    ...    ...   
29995     220000    0          3         1          0      0      0      0   
29996     150000    0          3         2          0     -1     -1     -1   
29997      30000    0          2         2          0      4      3      2   
29998      80000    0          3         1          0      1     -1      0   
29999      50000    0          2         1          0      0      0      0   

       PAY_4  PAY_5  ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_

In [24]:
### set sensitive attribute equal to 'SEX' or 'Age Group' ###

sen_att = 'Age Group'

In [4]:
#split data in a test and train set

from sklearn.model_selection import train_test_split
X = taiwan_data.loc[:, taiwan_data.columns != 'Creditworthiness']
y = taiwan_data.loc[:, taiwan_data.columns == 'Creditworthiness']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify = y, random_state=0)

#apply SMOTE to the trainingset
from imblearn.over_sampling import SMOTE
os = SMOTE(random_state=0)

columns = X_train.columns
os_data_X, os_data_y=os.fit_sample(X_train, y_train)
os_data_X = pd.DataFrame(data=os_data_X,columns=columns )
os_data_y= pd.DataFrame(data=os_data_y,columns=['Creditworthiness'])


In [5]:
# we can Check the numbers of our data
print("length of oversampled data is ",len(os_data_X))
print (len(os_data_y))
print("Number of no approval in oversampled data",len(os_data_y[os_data_y['Creditworthiness']==0]))
print("Number of approval",len(os_data_y[os_data_y['Creditworthiness']==1]))
print("Proportion of no approval data in oversampled data is ",len(os_data_y[os_data_y['Creditworthiness']==0])/len(os_data_X))
print("Proportion of approval data in oversampled data is ",len(os_data_y[os_data_y['Creditworthiness']==1])/len(os_data_X))

length of oversampled data is  37382
37382
Number of no approval in oversampled data 18691
Number of approval 18691
Proportion of no approval data in oversampled data is  0.5
Proportion of approval data in oversampled data is  0.5


In [6]:
#ranker 

from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier

#Create KNN Classifier
knn = KNeighborsClassifier(n_neighbors = 15)
knn.fit(os_data_X, os_data_y)

from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
y_pred = knn.predict(os_data_X)


from sklearn.metrics import classification_report
print(classification_report(os_data_y, y_pred))



<ipython-input-6-7db6ffd7c207>:8: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  knn.fit(os_data_X, os_data_y)


              precision    recall  f1-score   support

           0       0.70      0.86      0.77     18691
           1       0.82      0.63      0.71     18691

    accuracy                           0.75     37382
   macro avg       0.76      0.75      0.74     37382
weighted avg       0.76      0.75      0.74     37382



In [7]:
matrix = sklearn.metrics.confusion_matrix(os_data_y, y_pred)
sklearn.metrics.accuracy_score(os_data_y, y_pred)
print(matrix)

[[16138  2553]
 [ 6889 11802]]


In [8]:
#result = os_data_X.append(os_data_y, ignore_index=True, sort=False)
result = pd.concat([os_data_X, os_data_y.reindex(os_data_X.index)], axis=1)
prob  = knn.predict_proba(os_data_X)[:,1]


result['prob'] = prob
print(result)
#df_test = [os_data_X, os_data_y, df_prob]

       LIMIT_BAL  SEX  EDUCATION  MARRIAGE  Age Group  PAY_0  PAY_2  PAY_3  \
0         420000    1          2         2          0      0      0      0   
1         230000    1          5         1          0     -1     -1     -1   
2          30000    1          2         1          0      2      2      2   
3          30000    1          2         1          1      2      2      2   
4         390000    1          1         1          0     -1      0     -1   
...          ...  ...        ...       ...        ...    ...    ...    ...   
37377      20000    1          3         1          0      1     -2     -2   
37378      10000    0          2         2          0      0      0      0   
37379     300000    0          1         1          0     -1     -1     -1   
37380      80000    0          2         2          0      2      2      2   
37381      20000    0          1         2          0     -1      0      0   

       PAY_4  PAY_5  ...  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_A

In [9]:
result_sorted = result.sort_values(by=[sen_att , 'prob'], ascending=[True, False], ignore_index=True)
result_sorted["action"] = pd.Series([0 for x in range(len(result_sorted.index))])

result_sorted


,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,Age Group,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,Creditworthiness,prob,action
0,170000,0,2,1,0,0,0,0,0,0,...,51269,1675,1500,1007,0,51269,1865,1,1.0,0
1,170000,0,2,2,0,0,0,0,0,0,...,64218,1300,2000,1000,2000,52292,2200,1,1.0,0
2,190000,0,1,2,0,0,0,0,0,0,...,113386,6112,4202,4200,4200,5000,4200,1,1.0,0
3,250000,0,1,2,0,0,0,0,0,0,...,168505,6845,5945,5919,6000,6059,5837,1,1.0,0
4,320000,0,1,1,0,0,0,0,0,0,...,118228,4028,4000,5000,5000,5000,5000,1,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37377,50000,1,1,2,0,1,1,1,1,1,...,39720,1930,1522,1792,1368,1415,307,0,0.0,0
37378,30000,1,2,1,0,0,0,0,2,0,...,16912,1333,2166,1166,833,2499,750,0,0.0,0
37379,144127,1,1,1,0,2,0,0,0,0,...,52100,1479,2507,2264,1728,1941,1897,0,0.0,0
37380,20000,1,2,2,0,2,2,7,7,7,...,1800,0,0,0,0,0,0,0,0.0,0


In [10]:

S = sen_att
y = 'Creditworthiness'
sv = 1
nv=  0
df=  result_sorted 
DP = 0
DN = 0
FP = 0
FN = 0
    
for row in range(len(df)) :
    if df.at[row, S] == 1 and df.at[row, y] == 0:
        DN +=1
    elif df.at[row, S] == 1 and df.at[row, y] == 1: 
        DP += 1 
    elif df.at[row, S] == 0 and df.at[row, y] == 0: 
        FN += 1
    else:
        FP += 1 

print(DP, DN, FP, FN )
    #identify needed group sizes
    
df_size = DN + DP + FN + FP
size_group = (df_size/4)
    
print(DN, size_group)
    

11477 7699 7214 10992
7699 9345.5


In [11]:
##### if DN is smaller than size_group, no changes are made ####

df["action"] = pd.Series([0 for x in range(len(df.index))])
    
sort_DN = df.sort_values(by=[ S, 'prob'], ascending=[False, False], ignore_index=True)
dn_count = DN

if DN > size_group:
    for row in range(len(sort_DN)):
        if dn_count > size_group and sort_DN.at[row, S] == sv and sort_DN.at[row, y] == 0:
            sort_DN.at[row, "action"] == 2
            dn_count -= 1
        else:
            pass
        #return sort_DN
    else: 
        pass
    #print(dn_count, DN, size_group)
print (sort_DN)


       LIMIT_BAL  SEX  EDUCATION  MARRIAGE  Age Group  PAY_0  PAY_2  PAY_3  \
0         230000    1          5         1          0     -1     -1     -1   
1         610000    1          1         2          0      0      0      0   
2         320000    1          2         2          0     -1     -1     -1   
3          20000    1          3         2          0     -1     -1     -1   
4         190000    1          3         2          0      0      0      0   
...          ...  ...        ...       ...        ...    ...    ...    ...   
37377      20000    0          2         1          0      0      0      0   
37378     150000    0          1         1          0      2      2      2   
37379      20000    0          1         2          0      3      2      2   
37380      80000    0          1         1          0     -1     -1     -1   
37381      80000    0          1         2          0      0      0      2   

       PAY_4  PAY_5  ...  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AM

In [12]:
sort_DN['action'].value_counts()

0    37382
Name: action, dtype: int64

In [13]:

sort_DP = sort_DN.sort_values(by=[S, 'prob'], ascending=[False, True], ignore_index=True)

  
if DP < size_group:
    dp_count = DP
    for row in range(len(sort_DP)):
        if dp_count < size_group and sort_DP.at[row, S] == sv and sort_DP.at[row, y] == 1:
            sort_DP.at[row, "action"] = 1
                #print(sort_DP.at[row, "action"])
            dp_count += 1
        else:
            pass
print(sort_DP)
    
    
 

       LIMIT_BAL  SEX  EDUCATION  MARRIAGE  Age Group  PAY_0  PAY_2  PAY_3  \
0          20000    1          3         2          1      2      0      0   
1         290000    1          1         1          0     -1     -1     -1   
2          50000    1          2         2          0      1      3      3   
3          30000    1          2         2          0      2      2      2   
4          80000    1          1         2          0      0      0      2   
...          ...  ...        ...       ...        ...    ...    ...    ...   
37377     450000    0          2         1          0      0      0      0   
37378     500000    0          2         2          0      0      0      0   
37379     320000    0          2         1          0      0      0      0   
37380     160000    0          2         2          0      0      0      0   
37381     500000    0          1         1          0     -2     -2     -2   

       PAY_4  PAY_5  ...  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AM

In [14]:
sort_DP['action'].value_counts()

0    37382
Name: action, dtype: int64

In [15]:
sort_FN = sort_DP.sort_values(by=[sen_att, 'prob'], ascending=[True, False], ignore_index=True) 
            
if FN < size_group:
    fn_count = FN
        
    for row in range(len(sort_FN)):
        if dn_count < size_group and sort_FN.at[row, S] == nv and sort_FN.at[row, y] == 0:
            sort_FN.at[row, "action"] = 1
            fn_count += 1
        else: 
            pass
    else: 
        pass



In [16]:

sort_FP = sort_FN.sort_values(by=[sen_att, 'prob'], ascending=[True, True], ignore_index=True)
    
if FP > size_group: 
    fp_count = FP
    for row in range(len(sort_FP)):
        if fp_count > size_group and sort_FP.at[row, S] == nv and sort_FP.at[row, y] == 1:
            sort_FP.at[row, "action"] = 2
            fp_count -= 1
        else:
            pass 
            

    else:
        pass 
        


    
## 1 is duplicate 2 is delete     

In [17]:
print(sort_FP)
sort_FP['action'].value_counts()

       LIMIT_BAL  SEX  EDUCATION  MARRIAGE  Age Group  PAY_0  PAY_2  PAY_3  \
0         120000    0          1         1          0     -1     -1     -1   
1          10000    0          3         2          0      3      2      2   
2          30000    0          2         1          0      2      2      2   
3          20000    0          3         1          0      0      0      2   
4          80000    0          2         1          1      0      0      0   
...          ...  ...        ...       ...        ...    ...    ...    ...   
37377      90000    1          2         2          0     -2     -2     -2   
37378     280000    1          2         2          0     -2     -2     -1   
37379     280000    1          2         2          0      0      0      2   
37380     280000    1          2         1          0      2      2      2   
37381     360000    1          1         2          0     -1     -1     -1   

       PAY_4  PAY_5  ...  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AM

0    37382
Name: action, dtype: int64

In [18]:
df_duplicate = sort_FP
df_duplicate = df_duplicate.drop(df_duplicate[df_duplicate.action == 2].index)
df_duplicate = df_duplicate.drop(df_duplicate[df_duplicate.action == 0].index)
print (df_duplicate)

Empty DataFrame
Columns: [LIMIT_BAL, SEX, EDUCATION, MARRIAGE, Age Group, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6, BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6, PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6, Creditworthiness, prob, action]
Index: []

[0 rows x 26 columns]


In [19]:
sort_FP.append(df_duplicate)
final_dataset = sort_FP.drop(df_duplicate[df_duplicate.action == 2].index)
final_dataset = final_dataset.drop(columns=['prob', 'action'])
print(final_dataset)

       LIMIT_BAL  SEX  EDUCATION  MARRIAGE  Age Group  PAY_0  PAY_2  PAY_3  \
0         120000    0          1         1          0     -1     -1     -1   
1          10000    0          3         2          0      3      2      2   
2          30000    0          2         1          0      2      2      2   
3          20000    0          3         1          0      0      0      2   
4          80000    0          2         1          1      0      0      0   
...          ...  ...        ...       ...        ...    ...    ...    ...   
37377      90000    1          2         2          0     -2     -2     -2   
37378     280000    1          2         2          0     -2     -2     -1   
37379     280000    1          2         2          0      0      0      2   
37380     280000    1          2         1          0      2      2      2   
37381     360000    1          1         2          0     -1     -1     -1   

       PAY_4  PAY_5  ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler

X_train = final_dataset.loc[:, final_dataset.columns != 'Creditworthiness']
y_train = final_dataset.loc[:, final_dataset.columns == 'Creditworthiness']




logit = LogisticRegression(penalty='l2', solver = 'liblinear', max_iter = 5000)
#logit = LogisticRegression()
logit.fit(X_train, y_train)


y_pred =  logit.predict(X_test)


from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

matrix = sklearn.metrics.confusion_matrix(y_test, y_pred)
accuracy_score = sklearn.metrics.accuracy_score(y_test, y_pred)
print(matrix, accuracy_score)


C:\Users\Majda\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


              precision    recall  f1-score   support

           0       0.33      0.36      0.34      1327
           1       0.81      0.79      0.80      4673

    accuracy                           0.69      6000
   macro avg       0.57      0.58      0.57      6000
weighted avg       0.71      0.69      0.70      6000

[[ 484  843]
 [ 995 3678]] 0.6936666666666667


In [21]:
privileged_groups = [{sen_att: 0}]
unprivileged_groups = [{sen_att: 1}]
cost_constraint = "fnr"
randseed = 12345679 

In [22]:
fair_test_df = pd.concat([X_test, y_test], axis=1, join="inner")

In [23]:
# Metrics function
from collections import OrderedDict
from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

dataset = StandardDataset(fair_test_df, 
                          label_name='Creditworthiness', 
                          favorable_classes=[1], 
                          protected_attribute_names=[sen_att], 
                          privileged_classes=[[0]])

def fair_metrics(dataset, y_pred, disp = True):
    dataset_pred =dataset.copy()
    dataset_pred.labels = y_pred
        
    attr = dataset_pred.protected_attribute_names[0]
    
    idx = dataset_pred.protected_attribute_names.index(attr)
    privileged_groups =  [{attr:dataset_pred.privileged_protected_attributes[idx][0]}] 
    unprivileged_groups = [{attr:dataset_pred.unprivileged_protected_attributes[idx][0]}] 

    classified_metric_pred = ClassificationMetric(dataset, dataset_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

    metric_pred = BinaryLabelDatasetMetric(dataset_pred, unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)

    metrics = OrderedDict()
    metrics["Balanced accuracy"] = 0.5*(classified_metric_pred.true_positive_rate()+
                                             classified_metric_pred.true_negative_rate())
    metrics["Statistical parity difference"] = classified_metric_pred.statistical_parity_difference()
    metrics["Disparate impact"] = classified_metric_pred.disparate_impact()
    metrics["Average odds difference"] = classified_metric_pred.average_odds_difference()
    metrics["Equal opportunity difference"] = classified_metric_pred.equal_opportunity_difference()
    metrics["Theil index"] = classified_metric_pred.theil_index()
        
    if disp:
        for k in metrics:
            print("%s = %.4f" % (k, metrics[k]))


fair_metrics(dataset, y_pred)

Balanced accuracy = 0.5759
Statistical parity difference = 0.0190
Disparate impact = 1.0256
Average odds difference = 0.0062
Equal opportunity difference = 0.0149
Theil index = 0.2255
